In [1]:
#!pip3 install http://download.pytorch.org/whl/cpu/torch-1.0.0-cp36-cp36m-linux_x86_64.whl

!pip install fastai==1.0.61
!pip install torch==1.9.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

     |████████████████████████████████| 239 kB 524 kB/s            
  Preparing metadata (setup.py) ... - done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19191 sha256=90cfdeeb1cf4bf1a41542ab3707c405649aa56bccc293813f1646543d692a209
  Stored in directory: /root/.cache/pip/wheels/df/99/da/c34f202dc8fd1dffd35e0ecf1a7d7f8374ca05fbcbaf974b83
Successfully built nvidia-ml-py3
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.0.1
    Uninstalling typing-extensions-4.0.1:
      Successfully uninstalled typing-extensions-4.0.1
  Attempting uninstall: fastai
    Found existing installation: fastai 2.5.3
    Uninstalling fastai-2.5.3:
      Successfully uninstalled fastai-2.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem

In [2]:
!pip install sentencepiece

In [3]:
from fastai.text import *
import numpy as np
#import demoji
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb
from sklearn.utils import shuffle
from nltk.corpus import words
from bs4 import BeautifulSoup
import nltk
nltk.download('words')
import nltk, string, re, spacy,unicodedata, random
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.61', '1.9.0+cu102')

In [5]:
torch.cuda.set_device(0)

In [6]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [7]:
random_seed(42, True)

In [8]:
# Train Data
df_train_en = pd.read_csv('../input/tamilmisodatanew/ta-en-misogyny-train.csv', sep='\t', header=None)
df_train_ta = pd.read_csv('../input/tamilmisodatanew/ta-misogyny-train.csv', sep='\t', header=None)

# Dev Data
df_dev_en = pd.read_csv('../input/tamilmisodatanew/ta-en-misogyny-dev.csv', sep='\t', header=None)
df_dev_ta = pd.read_csv('../input/tamilmisodatanew/ta-misogyny-dev.csv', sep='\t', header=None)

# Test Data
df_test_en = pd.read_csv('../input/tamilmisodatanew/ta-en-misogyny-test.csv', sep='\t', header=None)
df_test_ta = pd.read_csv('../input/tamilmisodatanew/ta-misogyny-test.csv', sep='\t', header=None)

In [9]:
path = Path('./')

In [10]:
!pwd

/kaggle/working


In [11]:
cat2ind = {'None-of-the-above': 0,
 'Transphobic': 1,
 'Counter-speech': 2,
 'Misandry': 3,
 'Homophobia': 4,
 'Hope-Speech': 5,
 'Xenophobia': 6,
 'Misogyny': 7,
 'Not-Tamil': 8}

ind2cat = dict()
for i in cat2ind:
    ind2cat[cat2ind[i]] = i

def restructure_df(df):
    txts = df[1].tolist()
    labs = df[0].tolist()
    labs_new = []
    for i in labs:
        labs_new.append(ind2cat[i])
    return pd.DataFrame({0:labs_new, 1:txts})

def over_under_sample(df, shuffle = True):
    def sample(rows, req_len):
        return [rows[i%len(rows)] for i in range(req_len)]

    if shuffle:
        df.sample(frac=1).reset_index(drop=True)
    data_dict = {'Misandry': [], 'Misogyny': [], 'Xenophobia': [], 'None-of-the-above': [], 'Hope-Speech': [], 'Counter-speech': [], 'Transphobic': [], 'Homophobia': [], 'Not-Tamil': []}

    for i in range(len(df)):
        data_dict[df[0][i]].append(list(df.iloc[i]))
    
    req_len = len(df)//len(data_dict)

    for lab in data_dict:
        data_dict[lab] = sample(data_dict[lab], req_len)
    df_rows = []
    for lab in data_dict:
        df_rows += data_dict[lab]
    df_new = pd.DataFrame(df_rows, columns = list(df.columns))
    if shuffle:
        df_new = df_new.sample(frac=1).reset_index(drop=True)
    return df_new

In [12]:
# df_train.shape, df_valid.shape, df_test.shape

In [13]:
df_train = pd.DataFrame({0: df_train_ta[0].tolist() +df_train_en[0].tolist(), 1: df_train_ta[1].tolist() +df_train_en[1].tolist()})
df_valid = df_dev_ta
df_test = df_test_ta

In [14]:
df_train

,0,1
0,Hope-Speech,உங்கள் பேச்சை எதிர்பார்த்தேன். நல்ல விளக்கம் ...
1,None-of-the-above,எச். ராசாவால் இராமருக்கே இழிவு. இவர் எல்லாம் ...
2,None-of-the-above,கல்யாணம்னு வரும்போது தான் சாதி முக்கியம் எங்க...
3,None-of-the-above,Tentu Kottai ஐயா நான் தமிழ் இந்து.
4,Homophobia,Tamil selvan அடே தேவடியாவுக்கு பிரந்தவனே பச்ச...
...,...,...
8183,None-of-the-above,Noq day having no caste no religious certificate
8184,None-of-the-above,Mimicry escape aitaru bro. 😂
8185,None-of-the-above,Rajesh R u age u from
8186,Counter-speech,Bro unga videos nallarukku but don't do prank ...


In [15]:
df_train = shuffle(df_train, random_state = 10).reset_index(drop=True)
df_valid = shuffle(df_valid, random_state = 10).reset_index(drop=True)
df_test = shuffle(df_test, random_state = 10).reset_index(drop=True)

df_train[1] = df_train[1].apply(str)
df_valid[1] = df_valid[1].apply(str)
df_test[1] = df_test[1].apply(str)

In [16]:
def deEmojify(string):
    emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                              u"\U00002500-\U00002BEF"  # chinese char
                              u"\U00002702-\U000027B0"
                              u"\U00002702-\U000027B0"
                              u"\U000024C2-\U0001F251"
                              u"\U0001f926-\U0001f937"
                              u"\U00010000-\U0010ffff"
                              u"\u2640-\u2642"
                              u"\u2600-\u2B55"
                              u"\u200d"
                              u"\u23cf"
                              u"\u23e9"
                              u"\u231a"
                              u"\ufe0f"  # dingbats
                              u"\u3030"
                              "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def preprocess(text):
    text = deEmojify(text) #convert emojis to their defns in words, they might be useful
    text = re.sub(r'([\.\'\"\/\-\_\--])',' ', text) # remove punctuations , removes @USER / some abbreviatins
    to_remove_url = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
      '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = re.sub(to_remove_url,'',text)  # remove url patterns
    text = re.sub(" \d+", " ", text)
    text = text.replace(","," ")
    text = re.sub(r'(?:^| )\w(?:$| )', ' ', text).strip()
    punctuation='!!"$%&()*+-/:;<=>?[\\]^_{|}~.'
    text = ''.join(ch for ch in text if ch not in set(punctuation))
    # text = text.translate(str.maketrans('', '', string.punctuation))
    text = BeautifulSoup(text, 'html.parser').get_text()
    # Stopword Removing
    tokenizer = ToktokTokenizer()
    # convert sentence into token of words
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    text = ' '.join(ch for ch in tokens)
    return text 

def clean(df):
    df[1] = df[1].apply(lambda x: preprocess(x))

clean(df_train)
clean(df_valid)
clean(df_test)

In [17]:
df_train[df_train[0].isnull()].shape, df_valid[df_valid[0].isnull()].shape, df_test[df_test[0].isnull()].shape

((0, 2), (0, 2), (0, 2))

In [18]:
class TamilTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('../input/spm-sota-models/tamil_spm_8k.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [19]:
sp = spm.SentencePieceProcessor()
sp.Load(str('../input/spm-sota-models/tamil_spm_8k.model'))
itos = [sp.IdToPiece(int(i)) for i in range(8000)]

In [20]:
tamil_vocab = Vocab(itos)

In [21]:
tokenizer = Tokenizer(tok_func=TamilTokenizer, lang='ta')

In [22]:
label_cols = [0]

In [23]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=tamil_vocab)

/opt/conda/lib/python3.7/site-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


In [24]:
data_lm.show_batch()

idx,text
0,a ▁ vi t tu ▁ va ik a ▁ ma at ing a ▁ d a ▁ x x b os ▁ li ke ▁ v j ▁ an na ▁ x x b os ▁உண்மை ▁சொன்ன தற்கு ▁நன் றி ▁ x x b os ▁ஏ ண்டா ▁உ ங்களுக்கு ▁பேச ▁வேற ▁மத மே ▁இல்லை யா ▁இந்துக் கள் ▁என்ன ▁ஏ மா ளி யா ▁ராஜா ▁பேச ியது ▁சரி யே ▁ x
1,tu ▁ han d s ome ▁ ah ▁ ir uk ing a ▁ x x b os ▁வீட்டில் ▁பெண்கள் ▁இல்லை ▁என்ன ▁செய்வது ▁ x x b os ▁சாதி ▁வெ றி ▁காட்டு மி ரா ண்டி ▁நா யே ▁உ ன க்கு ▁சாதி ▁தவிர ▁வேற ▁என்ன ▁இரு க்கு ▁டா ▁பொ ட்ட ▁பு ண்டை ▁ x x b os ▁அ ய்யா ▁நீ ங்கள் ▁பேச ியது ▁நல்ல ▁மணி த த்துவம் ▁ x x b
2,an p adu ▁ ka la ach ar am ▁p al ak av al ak am ▁ sa ap ad a ▁ ma ath un ath u ▁ ma ath i ri ▁ el ath ai y um ▁ ma ath ir ath ing a ▁ d a ▁ ma at uk u ▁ ep di ▁ ka la ach ara ma ▁p a ak ur ing al a ▁
3,ai ▁ y a ▁p or ag al an u ▁ x x b os ▁ <unk> se en u <unk> <unk> ow th am <unk> ▁a ma ▁a ma ▁n an b a <unk> <unk> van ung a ▁ va z n th u ▁ an ub a vi ch ut u ▁ na ma la ▁ va z ha ▁ vi d am a ▁p and ran ga
4,முத்து ▁பேச லாம் ▁என்ன ங்க ▁சீ மான் ▁இப் டி ▁பண ் ற ீ ங்க ▁போய் ▁வைர முத்து ▁கி ட்ட ▁கேட்டு டு வா ங்க ▁அ ப்புற ம் ▁பேச ிக்க லாம் ▁ x x b os ▁ la ▁p es ur a ▁ g ir l ▁ k u <unk> ▁c or on a ▁ ir uk u ▁ po la ▁ x x b os <unk> <unk> ir sh <unk> <unk>


In [25]:
awd_lstm_config = awd_lstm_lm_config.copy()
awd_lstm_config['n_hid'] = 1150
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, config=awd_lstm_config, pretrained=False)

In [26]:
learn.load('/kaggle/input/pretrained-sota-models/wikitalm_8k_447_third')

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (8188 items)
x: LMTextList
▁ x x b os <unk> ▁m a y i z h <unk> <unk> e l a n ▁p on ▁n am er ▁s ent ▁ me,▁ x x b os ▁ x m s r k s ▁ j i ▁ ent ha ▁ ch ur ch ▁n u ▁s ol l ung a,▁ x x b os ▁ ed ha y um ▁ e van ung a ▁ vi t tu ▁ va ik a ▁ ma at ing a ▁ d a,▁ x x b os ▁ li ke ▁ v j ▁ an na,▁ x x b os ▁உண்மை ▁சொன்ன தற்கு ▁நன் றி
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (560 items)
x: LMTextList
▁ x x b os ▁சோ ம் பே றி கள் ▁விரும்பி ▁வர த ட் ச ணை,▁ x x b os ▁சி மான் ▁கே வள மான ▁ஒ று ▁மனிதன் ▁ப ச் சோ ந்த ▁நாய் ▁வைர முத்து ▁ஒ று ▁கட்டு ▁பன்றி,▁ x x b os ▁சம த்துவம் ▁பேசும் ▁ஆன ந் சீ னி வா சன் ▁சா ருக்கு ▁நன் றி,▁ x x b os ▁p r iya ▁p r iya ▁அர வாணி ▁பு ண்ட ▁எ ங்க டா ▁ ங் ங ோ த்தா ▁கூ தி ▁கி ழி யு தா,▁ x x b os ▁பெ சி க்கொ ன் டி று க்கும் பொ து ▁பின்ன ால ▁இறுக்க ிற வ ங்க ல ▁பார்த்து ▁பார்த்து ▁பேச ுகிறார் ▁அ ப்போ ▁முன் னா டி ▁ஆட் களே ▁இல்ல யா
y: LMLabelList
,,,,
Path: .;

Test: LabelList (700 items)
x: LMTextList
▁ x x b os <unk> ▁a b 

In [27]:
df_train = df_train.drop([0]).reset_index().drop(["index"], axis=1)
df_train[0].value_counts()

None-of-the-above    5015
Misandry             1276
Counter-speech        497
Xenophobia            392
Misogyny              336
Hope-Speech           299
Homophobia            207
Transphobic           163
Not-Tamil               2
Name: 0, dtype: int64

In [28]:
df_valid[0].value_counts()

None-of-the-above    346
Misandry             104
Counter-speech        36
Xenophobia            29
Misogyny              24
Hope-Speech           11
Homophobia             8
Transphobic            2
Name: 0, dtype: int64

In [29]:
df_test[0].value_counts()

None-of-the-above    417
Misandry             127
Misogyny              48
Counter-speech        47
Hope-Speech           26
Xenophobia            25
Homophobia             8
Transphobic            2
Name: 0, dtype: int64

In [30]:
learn.freeze()

In [31]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.718938,4.569289,0.278181,00:09


In [32]:
learn.unfreeze()

In [33]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.395640,4.412742,0.293248,00:11
1,3.192381,4.244159,0.316741,00:11
2,3.030974,4.181168,0.323605,00:11
3,2.924587,4.170158,0.328237,00:11
4,2.881798,4.167700,0.328571,00:12


In [34]:
learn.predict('▁தனது ▁சொந்த',n_words=10)

'▁தனது ▁சொந்த ▁அனுபவ ம் ▁அ சிங்க மான து ▁ x x b'

In [35]:
learn.save_encoder('fine_tuned_enc')

In [36]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=tamil_vocab, label_cols=label_cols, bs=64)

/opt/conda/lib/python3.7/site-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [37]:
data_clas.show_batch()

text,target
▁ x x b os <unk> ▁s w ath i <unk> ▁s ank ar ra j ▁c ore ect ▁ ip o ▁ ne en ga le y ▁ o th uk ut ing a ▁p ud h u ▁ ed ath uk u ▁ po na ▁ ka ka ▁b ay am a ▁ er uk um ▁n u ▁ ad he y ▁ min d s et ▁,Counter-speech
▁ x x b os <unk> <unk> van ga la <unk> <unk> id a ▁the li va ▁ y a ara al ay um ▁ cl ear ▁ ex p lan ation ▁ k ud uk ka ▁m id iya ad h u <unk> <unk> r uth an ▁in n or uth ana ▁ adi ch u ▁ k ol ran <unk> <unk> d ha <unk> <unk> id a ▁,Misandry
▁ x x b os <unk> <unk> aj a ▁ mo ha me d ▁ ne en ga ▁m ad ha ▁ sa ar b a ▁ pe y s ur e en ga ▁n an ▁ ad ha ▁m ad hi k ir en <unk> <unk> n ag al uk u ▁ or u ▁a an um ▁a an um ▁s er ur ad h u ▁ ep di,Counter-speech
▁ x x b os <unk> ▁s ex ▁ va ch ik ur ath uk u ▁m att um ▁ d han ▁ or u ▁p on n um ▁ or u ▁p ai y an um ▁ on n u ▁ se ran um a ▁ ap ram ▁in ga ▁ lo ve ▁ en ga ▁ ir uk u ▁ pl s ▁p pl ▁ g r ow,None-of-the-above
▁ x x b os <unk> ▁p o ▁ di ▁ lo os u ▁ k ood hi ▁s ug an y a ▁ y en d ra ▁m ot ta i ▁s un ni ▁ om b od h u ▁ th ay vi d y a a ▁p ay al ay <unk> ▁m o ot hi ra ▁s att i ▁ k an na d a ▁ th,Misogyny


In [38]:
del awd_lstm_config['tie_weights']
del awd_lstm_config['out_bias']

In [39]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5, config=awd_lstm_config)

In [40]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (8187 items)
x: TextList
▁ x x b os ▁ x m s r k s ▁ j i ▁ ent ha ▁ ch ur ch ▁n u ▁s ol l ung a,▁ x x b os ▁ ed ha y um ▁ e van ung a ▁ vi t tu ▁ va ik a ▁ ma at ing a ▁ d a,▁ x x b os ▁ li ke ▁ v j ▁ an na,▁ x x b os ▁உண்மை ▁சொன்ன தற்கு ▁நன் றி,▁ x x b os ▁ஏ ண்டா ▁உ ங்களுக்கு ▁பேச ▁வேற ▁மத மே ▁இல்லை யா ▁இந்துக் கள் ▁என்ன ▁ஏ மா ளி யா ▁ராஜா ▁பேச ியது ▁சரி யே
y: CategoryList
None-of-the-above,None-of-the-above,None-of-the-above,None-of-the-above,None-of-the-above
Path: .;

Valid: LabelList (560 items)
x: TextList
▁ x x b os ▁சோ ம் பே றி கள் ▁விரும்பி ▁வர த ட் ச ணை,▁ x x b os ▁சி மான் ▁கே வள மான ▁ஒ று ▁மனிதன் ▁ப ச் சோ ந்த ▁நாய் ▁வைர முத்து ▁ஒ று ▁கட்டு ▁பன்றி,▁ x x b os ▁சம த்துவம் ▁பேசும் ▁ஆன ந் சீ னி வா சன் ▁சா ருக்கு ▁நன் றி,▁ x x b os ▁p r iya ▁p r iya ▁அர வாணி ▁பு ண்ட ▁எ ங்க டா ▁ ங் ங ோ த்தா ▁கூ தி ▁கி ழி யு தா,▁ x x b os ▁பெ சி க்கொ ன் டி று க்கும் பொ து ▁பின்ன ால ▁இறுக்க ிற வ ங்க ல ▁பார்த்து ▁பார்த்து ▁பேச ுகிறார் ▁அ ப்போ ▁முன் னா

In [41]:
learn.freeze()

In [42]:
learn.loss_func.func

CrossEntropyLoss()

In [43]:
mcc = MatthewsCorreff()

In [44]:
f1 = FBeta(beta = 1,average = 'weighted')

In [45]:
learn.metrics = [f1]

In [46]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,f_beta,time
0,1.232277,1.114377,0.580403,00:05


/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


In [47]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,f_beta,time
0,1.070057,0.997211,0.634806,00:07


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

In [48]:
learn.unfreeze()
learn.fit_one_cycle(8, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='f_beta', name='final')])

epoch,train_loss,valid_loss,f_beta,time
0,0.902292,1.019348,0.619089,00:18
1,0.873637,1.026243,0.626823,00:14
2,0.856329,1.232639,0.607085,00:14
3,0.779987,1.080215,0.649932,00:15
4,0.701850,1.113537,0.638601,00:18
5,0.644573,1.176700,0.649337,00:18
6,0.619053,1.227636,0.645681,00:14
7,0.631276,1.252194,0.637481,00:17


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

Better model found at epoch 0 with f_beta value: 0.6190888285636902.


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

Better model found at epoch 1 with f_beta value: 0.6268231272697449.


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

Better model found at epoch 3 with f_beta value: 0.6499319076538086.


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

In [49]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]

i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key

df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result.head()

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


,query,actual_label,predicted_label,Misogyny,Misandry,Xenophobia,None-of-the-above,Hope-Speech,Transphobic,Not-Tamil,Counter-speech,Homophobia
0,Abirami Vijayakumar என் தலைவிதி என்று தான் நான...,None-of-the-above,None-of-the-above,0.003367,0.00255,0.010785,0.677732,0.250946,0.000536,0.000125,0.05348,0.00048
1,மாமனிதர் அண்ணன் நீங்கள் நீடூழி வாழவேண்டும்உங்க...,None-of-the-above,None-of-the-above,0.017506,0.029532,0.004924,0.835591,0.04261,0.001215,0.000019,0.06653,0.002072
2,சரியா சோலிங்கா சார்,None-of-the-above,None-of-the-above,0.017379,0.037679,0.008256,0.903365,0.002758,0.003207,0.000004,0.015977,0.011375
3,ஆமைக்கறி அண்ணன் சீமான்,None-of-the-above,None-of-the-above,0.025087,0.20183,0.015035,0.703507,0.008729,0.002877,0.000013,0.03389,0.009032
4,ஆனந்த சார் நியாயமான பேச்சு,None-of-the-above,None-of-the-above,0.005037,0.005389,0.011637,0.580042,0.342007,0.003388,0.000354,0.042541,0.009606


In [50]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.6457142857142857

In [51]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.3633268776107061

In [52]:
report = classification_report(df_result['actual_label'], df_result['predicted_label'],output_dict=True)

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
Counter-speech,0.333333,0.191489,0.243243,47.000000
Homophobia,0.714286,0.625000,0.666667,8.000000
Hope-Speech,0.357143,0.384615,0.370370,26.000000
Misandry,0.563492,0.559055,0.561265,127.000000
Misogyny,0.625000,0.312500,0.416667,48.000000
None-of-the-above,0.712500,0.820144,0.762542,417.000000
Transphobic,0.000000,0.000000,0.000000,2.000000
Xenophobia,0.000000,0.000000,0.000000,25.000000
accuracy,0.645714,0.645714,0.645714,0.645714
macro avg,0.413219,0.361600,0.377594,700.000000


In [54]:
df_result.to_csv('ULMFiT_ACD_TaEn_CD.csv', index=False)

In [55]:
import os
os.chdir(r'../working')
from IPython.display import FileLink
FileLink(r'ULMFiT_ACD_TaEn_CD.csv')

/kaggle/working/ULMFiT_ACD_TaEn_CD.csv